# DALL·E mini - Inference pipeline

*Generate images from a text prompt*

<img src="https://github.com/borisdayma/dalle-mini/blob/main/img/logo.png?raw=true" width="200">

This notebook illustrates [DALL·E mini](https://github.com/borisdayma/dalle-mini) inference pipeline.

Just want to play? Use [the demo](https://huggingface.co/spaces/flax-community/dalle-mini).

For more understanding of the model, refer to [the report](https://wandb.ai/dalle-mini/dalle-mini/reports/DALL-E-mini--Vmlldzo4NjIxODA).

## 🛠️ Installation and set-up

In [ ]:
# Install required libraries
!pip install -q transformers
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git
!pip install -q git+https://github.com/borisdayma/dalle-mini.git
!pip install -q wandb

We load required models:
* dalle·mini for text to encoded images
* VQGAN for decoding images
* CLIP for scoring predictions

In [ ]:
# Model references

# dalle-mini
DALLE_MODEL = 'dalle-mini/dalle-mini/model-262fxth3:latest'  # can be wandb artifact or 🤗 Hub or local folder
DALLE_COMMIT_ID = None  # used only with 🤗 hub

# VQGAN model
VQGAN_REPO = 'dalle-mini/vqgan_imagenet_f16_16384'
VQGAN_COMMIT_ID = 'e93a26e7707683d349bf5d5c41c5b0ef69b677a9'

# CLIP model
CLIP_REPO = 'openai/clip-vit-base-patch16'
CLIP_COMMIT_ID = None

In [ ]:
import jax
import jax.numpy as jnp

# type used for computation - use bfloat16 on TPU's
dtype = jnp.bfloat16 if jax.local_device_count() == 8 else jnp.float32

In [ ]:
# Load models & tokenizer
from dalle_mini.model import DalleBart
from vqgan_jax.modeling_flax_vqgan import VQModel
from transformers import AutoTokenizer, CLIPProcessor, FlaxCLIPModel
import wandb

# Load dalle-mini
if ':' in DALLE_MODEL:
    # wandb artifact
    artifact = wandb.Api().artifact(DALLE_MODEL)
    # we only download required files (no need for opt_state which is large)
    model_files = ['config.json', 'flax_model.msgpack', 'merges.txt', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.json']
    for f in model_files:
        artifact.get_path(f).download('model')
    model = DalleBart.from_pretrained('model', dtype=dtype, abstract_init=True)
    tokenizer = AutoTokenizer.from_pretrained('model')
else:
    # local folder or 🤗 Hub
    model = DalleBart.from_pretrained(DALLE_MODEL, revision=DALLE_COMMIT_ID, dtype=dtype, abstract_init=True)
    tokenizer = AutoTokenizer.from_pretrained(DALLE_MODEL, revision=DALLE_COMMIT_ID)

# Load VQGAN
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)

# Load CLIP
clip = FlaxCLIPModel.from_pretrained(CLIP_REPO, revision=CLIP_COMMIT_ID)
processor = CLIPProcessor.from_pretrained(CLIP_REPO, revision=CLIP_COMMIT_ID)

Model parameters are replicated on each device for faster inference.

In [ ]:
from flax.jax_utils import replicate

# convert model parameters for inference if requested
if dtype == jnp.bfloat16:
    model.params = model.to_bf16(model.params)

model_params = replicate(model.params)
vqgan_params = replicate(vqgan.params)
clip_params = replicate(clip.params)

Model functions are compiled and parallelized to take advantage of multiple devices.

In [ ]:
from functools import partial

# model inference
@partial(jax.pmap, axis_name="batch", static_broadcasted_argnums=(3,4))
def p_generate(tokenized_prompt, key, params, top_k, top_p):
    return model.generate(
        **tokenized_prompt,
        do_sample=True,
        num_beams=1,
        prng_key=key,
        params=params,
        top_k=top_k,
        top_p=top_p
    )

# decode images
@partial(jax.pmap, axis_name="batch")
def p_decode(indices, params):
    return vqgan.decode_code(indices, params=params)

# score images
@partial(jax.pmap, axis_name="batch")
def p_clip(inputs, params):
    logits = clip(params=params, **inputs).logits_per_image
    return logits

Keys are passed to the model on each device to generate unique inferences per device.

In [ ]:
import random

# create a random key
seed = random.randint(0, 2**32-1)
key = jax.random.PRNGKey(seed)

## 🖍 Text Prompt

Our model may require to normalize the prompt.

In [ ]:
from dalle_mini.text import TextNormalizer

text_normalizer = TextNormalizer() if model.config.normalize_text else None

Let's define a text prompt.

In [ ]:
prompt = 'view of the beach during sunset'

In [ ]:
processed_prompt = text_normalizer(prompt) if model.config.normalize_text else prompt

We repeat the prompt on each device and tokenize it.

In [ ]:
# repeat the prompt on each device
repeated_prompts = [processed_prompt] * jax.device_count()

# tokenize
tokenized_prompt = tokenizer(repeated_prompts, return_tensors='jax', padding='max_length', truncation=True, max_length=128).data
tokenized_prompt

Notes:

* `0`: BOS, special token representing the beginning of a sequence
* `2`: EOS, special token representing the end of a sequence
* `1`: special token representing the padding of a sequence when requesting a specific length

Finally we distribute the tokenized prompt onto the devices.

In [ ]:
from flax.training.common_utils import shard

tokenized_prompt = shard(tokenized_prompt)

## 🎨 Generate images

We generate images using dalle-mini model and decode them with the VQGAN.

In [ ]:
# number of predictions
n_predictions = 8

# We can customize top_k/top_p used for generating samples
gen_top_k = None
gen_top_p = None

In [ ]:
from flax.training.common_utils import shard_prng_key
import numpy as np
from PIL import Image
from tqdm.notebook import trange

# generate images
images = []
for i in trange(n_predictions // jax.device_count()):
    # get a new key
    key, subkey = jax.random.split(key)
    # generate images
    encoded_images = p_generate(tokenized_prompt, shard_prng_key(subkey), model_params, gen_top_k, gen_top_p)
    # remove BOS
    encoded_images = encoded_images.sequences[..., 1:]
    # decode images
    decoded_images = p_decode(encoded_images, vqgan_params)
    decoded_images = decoded_images.clip(0., 1.).reshape((-1, 256, 256, 3))
    for img in decoded_images:
        images.append(Image.fromarray(np.asarray(img * 255, dtype=np.uint8)))

Let's calculate their score with CLIP.

In [ ]:
# get clip scores
clip_inputs = processor(text=[prompt] * jax.device_count(), images=images, return_tensors='np', padding='max_length', max_length=77, truncation=True).data
logits = p_clip(shard(clip_inputs), clip_params)
logits = logits.squeeze()

Let's display images ranked by CLIP score.

In [ ]:
print(f'Prompt: {prompt}\n')
for idx in logits.argsort()[::-1]:
    display(images[idx])
    print(f'Score: {logits[idx]:.2f}\n')